# JAT Capstone Project (Week 3)
This notebook will be the main location for the Capstone Project

## Start of the Week 3 Assignment

In [9]:
import pandas as pd
import numpy as np

**Import the scraping library suggested**

In [3]:
import requests 
from bs4 import BeautifulSoup 

**Use the Notebook to build the code to scrape the following Wikipedia page, https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M, in order to obtain the data that is in the table of postal codes and to transform the data into a _pandas dataframe_**

In [15]:
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M" # From class instructions

web_page = requests.get(url).text #get page
web_page_result = BeautifulSoup(web_page, 'html.parser') #transform the text to html

neightborhood_table = web_page_result.find('table', class_ = 'wikitable')
neightborhood_data = neightborhood_table.find_all('tr')

# get the Postcode, Borough and Neighbourhood from the neightborhood_data
neightborhood_info = []
for each in neightborhood_data:
    res = each.text.split('\n')[1:-1] # remove some empty strings
    neightborhood_info.append(res)
    
neightborhood_info[0:5]

[['Postcode', 'Borough', 'Neighbourhood'],
 ['M1A', 'Not assigned', 'Not assigned'],
 ['M2A', 'Not assigned', 'Not assigned'],
 ['M3A', 'North York', 'Parkwoods'],
 ['M4A', 'North York', 'Victoria Village']]

**The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood**

In [18]:
neightborhood_info[0][-3] = 'PostalCode' # change the 1st column title to match instructions
neightborhood_info[0][-1] = 'Neighborhood' # change the 3rd column title to match instructions
neighborhood_df = pd.DataFrame(neightborhood_info[1:], columns=neightborhood_info[0])

neighborhood_df.head(5)

,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


**Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.  And if a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough.**

In [30]:
NA_boroughs = neighborhood_df.index[neighborhood_df['Borough'] == 'Not assigned']
NA_neighborhoods = neighborhood_df.index[neighborhood_df['Neighborhood'] == 'Not assigned']
 
for index in NA_neighborhoods:
    neighborhood_df['Neighborhood'][index] = neighborhood_df['Borough'][index]
    
neighborhood_df.drop(neighborhood_df.index[NA_boroughs], inplace=True) #Drop the NA Boroughs
neighborhood_df.reset_index(drop=True, inplace=True)

neighborhood_df.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights


**More than one neighborhood can exist in one postal code area.  These two rows will be combined into one row with the neighborhoods separated with a comma**

In [42]:
gbPC = neighborhood_df.groupby('PostalCode')

CD_neighborhoods = gbPC['Neighborhood'].apply(lambda x: ', '.join(x))
GRP_boroughs = gbPC['Borough'].apply(lambda x: set(x).pop())
CDF_df = pd.DataFrame(list(zip(GRP_boroughs.index, GRP_boroughs, CD_neighborhoods)))
CDF_df.columns = ['PostalCode', 'Borough', 'Neighborhood']

CDF_df.head(10)

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park"
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge"
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff, Cliffside West"


**In the last cell of your notebook, use the .shape method to print the number of rows of your dataframe.**

In [50]:
print('The number of rows in our dataframe is', CDF_df.shape[0])

The number of rows in our dataframe is 103


**Last step is to submit a link to your Notebook on your Github repository.**